# Tutorial 2: Mappings, masked channels, and POI models

Masked channels are auxiliary channels that don't enter the fit in the default setup (allthough they might indirectly through more advanced use cases such as regularization). They contain processes with systematic variations as regular channels. This allows to propagate the fitresult into these distributions.

A common use case is a cross section measurement. The cross section is not simply scaling with the signal strength modifier, i.e. $\sigma_\mathrm{measured} \neq \mu \sigma$. Instead, it depends on all systematic uncertainties that change the cross section, usually the theory uncertainties. Masked channels allow to define histograms containing the full dependency of the cross section, i.e. $\sigma_\mathrm{measured} = \sigma(\mu, \vec{\theta})$ where $\vec{\theta}$ are all systematic uncertainties that the cross section depends on. In case a differential cross section measurement is performed, the masked channel contains the generator-level distribution wit all its dependencies.

Mappings are transformations of parameters and observables (histograms). Baseline mappings are 'BaseMapping', 'Project', 'Ratio', or 'AngularCoefficients' but custom mappings can be defined. Any differential function can be implemented as a mapping. Mappings can be applied to regular or masked channels.

## Setting up the environment

Same as before

In [1]:
import os
import sys
from pathlib import Path

# 1. Emulate RABBIT_BASE (points to the 'rabbit' root)
# Since we are in rabbit/notebooks/, we go up one level
rabbit_base = str(Path(os.getcwd()).parent.absolute())
os.environ['RABBIT_BASE'] = rabbit_base

# 2. Update PYTHONPATH so you can 'import rabbit'
if rabbit_base not in sys.path:
    sys.path.append(rabbit_base)

# 3. Update PATH so you can run scripts from rabbit/bin/
bin_path = os.path.join(rabbit_base, 'bin')
if bin_path not in os.environ['PATH']:
    os.environ['PATH'] = bin_path + os.pathsep + os.environ['PATH']

print(f"RABBIT_BASE: {os.environ['RABBIT_BASE']}")

RABBIT_BASE: /home/david/work/Repos/rabbit


In [2]:
from IPython.display import Image, display, IFrame, HTML

## Generating a synthetic toy model

We generate the same model as in the tutorial 1, but now we an additional category axis, this could for example reflect 2 charges, lepton flavors, or any other variable. We also include a masked channel for the cross sections of the signal in these two categories.

In [14]:
import numpy as np
import hist

axis = hist.axis.Regular(10,0,1, name="x")
axis_cat = hist.axis.Regular(2,-2,2, name="cat")

# Create histograms for signal, 2 backhrounds, and data 
h_sig = hist.Hist(axis, axis_cat, storage=hist.storage.Weight())  
h_flat = hist.Hist(axis, axis_cat, storage=hist.storage.Weight())  
h_exp = hist.Hist(axis, axis_cat, storage=hist.storage.Weight())  
h_data = hist.Hist(axis, axis_cat, storage=hist.storage.Double())  
  
# Generate and fill components with weights  
np.random.seed(42)  
# Gaussian signal (mean=0.5, std=0.1) with weights  
sig_samples = np.random.normal(0.5, 0.1, 5000)
sig_cats = np.random.choice([-1, 1], size=5000)
sig_weights = np.random.normal(1.0, 0.2, 5000)  # Mean weight=1, sigma=0.2 
h_sig.fill(sig_samples, sig_cats, weight=sig_weights)  
  
# Flat background with weights  
flat_samples = np.random.uniform(0, 1, 4000)  
flat_cats = np.random.choice([-1, 1], size=4000)
flat_weights = np.random.normal(0.5, 0.1, 4000)  # Mean weight=0.5, sigma=0.1
h_flat.fill(flat_samples, flat_cats, weight=flat_weights)  
  
# Exponential background with weights  
exp_samples = np.random.exponential(0.2, 2000)  
exp_cats = np.random.choice([-1, 1], size=2000)
exp_weights = np.random.normal(1.5, 0.3, 2000)  # Mean weight=1.5, sigma=0.2 
h_exp.fill(exp_samples, exp_cats, weight=exp_weights)  
  
# Sum components and add Poisson fluctuations
h_data.values()[...] = np.random.poisson(  
    h_sig.values() + h_flat.values() + h_exp.values()  
)

# Input tensor construction

In [27]:
from rabbit import tensorwriter  

# 1. Initialize with the 2D channel
writer = tensorwriter.TensorWriter(systematic_type="log_normal")  

# Note: axis and axis_cat were defined in your previous step
writer.add_channel([axis, axis_cat], name="ch0")

# 2. Add Data and Processes (using the 2D histograms)
writer.add_data(h_data, "ch0")  
writer.add_process(h_flat, "flat_bkg", "ch0", signal=False)  
writer.add_process(h_exp, "exp_bkg", "ch0", signal=False)  
writer.add_process(h_sig, "signal", "ch0", signal=True)  

# 3. Normalization Systematics (unchanged, applies to the whole volume)
writer.add_norm_systematic("lumi", ["signal", "flat_bkg", "exp_bkg"], "ch0", 1.01)
writer.add_norm_systematic("flat_bkg_norm", "flat_bkg", "ch0", 1.05)

# 4. Generate and Add 2D Shape Systematics
def make_shape_var(h, factor_func):  
    h_var = h.copy()  
    
    # 1. Calculate the x-dependent weights (1D array)
    centers_x = h.axes[0].centers  
    weights_x = factor_func(centers_x)  
    
    # 2. Create the sign flip for the categories
    # Assuming bin 0 is 'negative' and bin 1 is 'positive'
    # This creates an array: [-1, 1]
    category_signs = np.array([-1, 1])
    
    # 3. Combine them using broadcasting
    # (N, 1) * (1, 2) results in an (N, 2) matrix of weights
    # The first column will be -weights_x, the second will be +weights_x
    total_weights = weights_x[:, np.newaxis] * category_signs[np.newaxis, :]
        
    # 4. Apply to the histogram values
    h_var.values()[...] = h.values() * (1 + total_weights)  
    return h_var

# Variation for flat background
h_flat_up = make_shape_var(h_flat, lambda c: 0.05 * np.exp(2 * (c - 0.5)))  
h_flat_dn = make_shape_var(h_flat, lambda c: -0.03 * np.exp(2 * (c - 0.5)))  

writer.add_systematic(  
    [h_flat_up, h_flat_dn],  
    "flat_bkg_shape",  
    "flat_bkg",  
    "ch0",  
    symmetrize="linear",
    constrained=True,  
)  

# Variation for signal (slope)
h_sig_up = make_shape_var(h_sig, lambda c: 0.5 * (c - 0.5))  
h_sig_dn = make_shape_var(h_sig, lambda c: -0.5 * (c - 0.5))  

writer.add_systematic(  
    [h_sig_up, h_sig_dn],  
    "slope",  
    "signal",  
    "ch0",  
    symmetrize="average",  
    constrained=False,  
)

In [30]:
# Add a masked channel
#   for simplicity we are skipping convolution effect between cross sections (gen-level) 
#   and fitted detector-level distributions

h_sig_xsec = hist.Hist(axis_cat, storage=hist.storage.Weight(), data=h_sig.project("cat"))

writer.add_channel([axis_cat], name="ch0_masked", masked=True)
writer.add_process(h_sig_xsec, "signal", "ch0_masked", signal=True)

writer.add_systematic(  
    [h_sig_up.project("cat"), h_sig_dn.project("cat")],  
    "slope",  
    "signal",  
    "ch0_masked",  
    symmetrize="average",  
    constrained=False,  
)

In [31]:
import os

directory="results/mappings_and_masked_channels/"
if not os.path.exists(directory):
    os.makedirs(directory)

# Write the input tensor to HDF5  
writer.write(
    outfilename=f"{directory}/input.hdf5"
)

### Masked channels

## Input data diagnostics

Plot inputdata
Debug inputdata

# Fit

## Fit Diagnostics

### Prefit & postfit plots

### Pulls and impacts

### Variations

## Mappings

### Baseline mappings

### Custom mappings

## POI Models

### Baseline POI models

### Custom POI model